# Задача классификации текста

Как уже упоминалось, мы сосредоточимся на простой задаче классификации текста на основе набора данных **AG_NEWS**, которая заключается в классификации заголовков новостей в одну из 4 категорий: Мир, Спорт, Бизнес и Наука/Технологии.

## Набор данных

Этот набор данных встроен в модуль [`torchtext`](https://github.com/pytorch/text), поэтому мы можем легко получить к нему доступ.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Здесь `train_dataset` и `test_dataset` содержат коллекции, которые возвращают пары метки (номер класса) и текста соответственно, например:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Итак, давайте выведем первые 10 новых заголовков из нашего набора данных:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Поскольку наборы данных являются итераторами, если мы хотим использовать данные несколько раз, нам нужно преобразовать их в список:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Токенизация

Теперь нам нужно преобразовать текст в **числа**, которые можно представить в виде тензоров. Если мы хотим получить представление на уровне слов, необходимо выполнить два шага:
* использовать **токенизатор** для разделения текста на **токены**
* создать **словарь** этих токенов.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Используя словарь, мы можем легко закодировать наш токенизированный строку в набор чисел:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Представление текста в виде "мешка слов"

Поскольку слова несут смысл, иногда можно понять содержание текста, просто взглянув на отдельные слова, независимо от их порядка в предложении. Например, при классификации новостей слова, такие как *погода*, *снег*, скорее всего, укажут на *прогноз погоды*, тогда как слова, такие как *акции*, *доллар*, будут относиться к *финансовым новостям*.

**Мешок слов** (Bag of Words, BoW) — это наиболее часто используемое традиционное векторное представление. Каждое слово связано с индексом вектора, а элемент вектора содержит количество вхождений слова в данном документе.

![Изображение, показывающее, как представление "мешка слов" хранится в памяти.](../../../../../translated_images/ru/bag-of-words-example.606fc1738f1d7ba9.webp)

> **Note**: Вы также можете представить BoW как сумму всех векторов с единичным кодированием (one-hot encoding) для отдельных слов в тексте.

Ниже приведен пример того, как сгенерировать представление "мешка слов" с использованием библиотеки Scikit Learn на Python:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Чтобы вычислить вектор мешка слов из векторного представления нашего набора данных AG_NEWS, мы можем использовать следующую функцию:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Примечание:** Здесь мы используем глобальную переменную `vocab_size` для указания размера словаря по умолчанию. Поскольку размер словаря часто бывает довольно большим, мы можем ограничить его наиболее часто встречающимися словами. Попробуйте уменьшить значение `vocab_size` и запустить код ниже, чтобы увидеть, как это влияет на точность. Вы можете ожидать некоторого снижения точности, но не драматического, в обмен на более высокую производительность.


## Обучение классификатора BoW

Теперь, когда мы научились создавать представление текста в виде мешка слов (Bag-of-Words), давайте обучим на его основе классификатор. Сначала нам нужно преобразовать наш набор данных для обучения таким образом, чтобы все позиционные векторные представления были преобразованы в представление мешка слов. Это можно сделать, передав функцию `bowify` в качестве параметра `collate_fn` стандартному `DataLoader` из библиотеки torch:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Теперь давайте определим простой классификатор нейронной сети, который содержит один линейный слой. Размер входного вектора равен `vocab_size`, а размер выхода соответствует количеству классов (4). Поскольку мы решаем задачу классификации, конечная функция активации — `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Теперь мы определим стандартный цикл обучения в PyTorch. Поскольку наш набор данных довольно большой, для учебных целей мы будем обучать только один эпоху, а иногда даже меньше одной эпохи (параметр `epoch_size` позволяет ограничить обучение). Мы также будем сообщать накопленную точность обучения во время обучения; частота отчетов задается с помощью параметра `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## Биграммы, триграммы и N-граммы

Одно из ограничений подхода "мешок слов" заключается в том, что некоторые слова являются частью многословных выражений. Например, слово "hot dog" имеет совершенно другое значение, чем слова "hot" и "dog" в других контекстах. Если мы всегда представляем слова "hot" и "dog" одними и теми же векторами, это может запутать нашу модель.

Чтобы решить эту проблему, **представления N-грамм** часто используются в методах классификации документов, где частота каждого слова, двухсловных или трехсловных выражений является полезной характеристикой для обучения классификаторов. В представлении биграмм, например, мы добавляем все пары слов в словарь, помимо оригинальных слов.

Ниже приведен пример того, как создать представление "мешок слов" с биграммами, используя Scikit Learn:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Основным недостатком подхода N-грамм является то, что размер словаря начинает расти чрезвычайно быстро. На практике необходимо сочетать представление Н-грамм с некоторыми методами уменьшения размерности, такими как *встраивания*, о которых мы поговорим в следующем разделе.

Чтобы использовать представление Н-грамм в нашем наборе данных **AG News**, нам нужно создать специальный словарь Н-грамм:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Мы могли бы использовать тот же код, что и выше, чтобы обучить классификатор, однако это было бы очень неэффективно с точки зрения памяти. В следующем разделе мы будем обучать биграммный классификатор, используя эмбеддинги.

> **Примечание:** Оставляйте только те n-граммы, которые встречаются в тексте больше указанного количества раз. Это гарантирует, что редкие биграммы будут исключены, и значительно уменьшит размерность. Для этого установите параметр `min_freq` на более высокое значение и наблюдайте, как изменяется длина словаря.


## Частотность термина и обратная частотность документа TF-IDF

В представлении BoW (мешок слов) все слова имеют одинаковый вес, независимо от их значимости. Однако очевидно, что часто встречающиеся слова, такие как *a*, *in* и т.д., гораздо менее важны для классификации, чем специализированные термины. На самом деле, в большинстве задач обработки естественного языка некоторые слова имеют большее значение, чем другие.

**TF-IDF** расшифровывается как **частотность термина – обратная частотность документа**. Это модификация мешка слов, где вместо бинарного значения 0/1, указывающего на наличие слова в документе, используется значение с плавающей точкой, связанное с частотой появления слова в корпусе.

Более формально, вес $w_{ij}$ слова $i$ в документе $j$ определяется как:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
где
* $tf_{ij}$ — количество появлений слова $i$ в документе $j$, то есть значение BoW, которое мы рассматривали ранее
* $N$ — количество документов в коллекции
* $df_i$ — количество документов, содержащих слово $i$ во всей коллекции

Значение TF-IDF $w_{ij}$ увеличивается пропорционально количеству раз, которое слово появляется в документе, и уменьшается в зависимости от количества документов в корпусе, содержащих это слово. Это помогает учитывать тот факт, что некоторые слова встречаются чаще других. Например, если слово появляется *в каждом* документе коллекции, $df_i=N$, и $w_{ij}=0$, такие термины полностью игнорируются.

Вы можете легко создать векторизацию текста с использованием TF-IDF через Scikit Learn:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Заключение

Хотя представления TF-IDF придают вес частоте различных слов, они не способны передавать значение или порядок. Как сказал известный лингвист Дж. Р. Фёрс в 1935 году: «Полное значение слова всегда контекстуально, и никакое изучение значения вне контекста нельзя воспринимать всерьез». Позже в этом курсе мы узнаем, как извлекать контекстуальную информацию из текста с помощью языкового моделирования.



---

**Отказ от ответственности**:  
Этот документ был переведен с помощью сервиса автоматического перевода [Co-op Translator](https://github.com/Azure/co-op-translator). Хотя мы стремимся к точности, пожалуйста, учитывайте, что автоматические переводы могут содержать ошибки или неточности. Оригинальный документ на его родном языке следует считать авторитетным источником. Для получения критически важной информации рекомендуется профессиональный перевод человеком. Мы не несем ответственности за любые недоразумения или неправильные интерпретации, возникающие в результате использования данного перевода.
